**Landscape Evolution Modeling of the Brushy Creek Suspected Impact Crater using Landlab.
The code is annotated in a reproducible way.
Will be useful to translate in similar settings worldwide, pending on optimal site-specific parameterization**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.warp import transform as rio_transform

from landlab import RasterModelGrid
from landlab.components import (
    FlowAccumulator, DepressionFinderAndRouter, FastscapeEroder, LinearDiffuser
)

# ============================================================
# USER SETTINGS 
# ============================================================

DEM_TIF = "output_USGS10m.tif"  # ORIGINAL, unflipped, georeferenced DEM

# Suspected crater center (decimal degrees)
crater_lat = 30.765955555555557
crater_lon = -90.73228333333334

# Reconstruction window
START_BP = 35000.0

# --- FAST coarse scan settings ---
impact_times_bp = np.arange(5000.0, START_BP + 1, 5000.0)
dt_years = 500.0

# Landscape process parameters (determined through iterative runs)
# Stream-power incision/erosion coefficient used by FastscapeEroder.
# K_sp_post > K_sp_pre, assuming the landscape became more erodible after the impact time 
# Linear hillslope diffusion coefficient used by LinearDiffuser.
# kappa_post > kappa_pre, assuming hillslope smoothing is stronger after impact time
K_sp_pre,  kappa_pre  = 3e-7, 0.005
K_sp_post, kappa_post = 1e-6, 0.015

# “Pre-impact” starting surface assumption
# Bigger sigma_pix_preimpact ⇒ more smoothing ⇒ more removal of small-scale modern drainage/incision.
sigma_pix_preimpact = 4.0

# Crater geometry grids
D_list = [1200.0, 1600.0, 1800, 2000.0]     # meters
depth_list = [5.0, 10.0, 15.0, 20.0]        # meters
rim_height_list = [2.0, 3.5, 4.5, 5.5]      # meters

# Controls the thickness of the rim ring, as a fraction of diameter.
# Bigger rim_rel_width ⇒ broader rim ring (more spread out, less sharp).
# soft larger ⇒ bowl drops more steeply near the rim and stays flatter near center (more “flat-floored” tendency).
# Try slightly varying the value
rim_rel_width = 0.15
soft = 2.5

# Mask radius multiplier (outside this radius we compute RMSE)
# it controls how much the crater neighborhood is allowed to influence the fit metric.
# Try 1.2-1.5
mask_mult = 1.5


# ============================================================
# 1) Read DEM + convert pixel size to meters (EPSG:4269 fix)
# ============================================================
with rasterio.open(DEM_TIF) as src:
    dem = src.read(1)              # 2D, row 0 = north (standard)
    transform = src.transform
    crs = src.crs
    nrows, ncols = dem.shape

print("DEM CRS:", crs)
print("DEM shape:", dem.shape)
print("Pixel size:", transform.a, transform.e)

if crs is None:
    raise ValueError("DEM CRS is None. Need CRS for crater placement from lat/lon.")

epsg = crs.to_epsg() if crs is not None else None

# --- Geographic CRS (degrees): handle BOTH EPSG:4326 and EPSG:4269 ---
if epsg in (4326, 4269):
    m_per_deg_lat = 111320.0
    m_per_deg_lon = 111320.0 * np.cos(np.deg2rad(crater_lat))
    dx_m = abs(transform.a) * m_per_deg_lon
    dy_m = abs(transform.e) * m_per_deg_lat

    print("Pixel size (deg):", transform.a, transform.e)
    print("Approx pixel size (m): dx_m =", dx_m, "dy_m =", dy_m)
else:
    # Projected CRS (typically meters; if feet, convert before using)
    dx_m = abs(transform.a)
    dy_m = abs(transform.e)
    print("Pixel size assumed in CRS units: dx =", dx_m, "dy =", dy_m)


# ============================================================
# 2) Lat/lon -> DEM pixel (row,col)
# ============================================================
x_map, y_map = rio_transform("EPSG:4326", crs, [crater_lon], [crater_lat])
x_map, y_map = x_map[0], y_map[0]

col_f, row_f = ~transform * (x_map, y_map)
row = int(np.round(row_f))
col = int(np.round(col_f))

print("Crater pixel in DEM (row,col):", row, col)
if not (0 <= row < nrows and 0 <= col < ncols):
    raise ValueError("Crater pixel is outside DEM extent.")


# ============================================================
# 3) Build Landlab grid 
# ============================================================
model_grid = RasterModelGrid((nrows, ncols), xy_spacing=(dy_m, dx_m))
model_grid.add_field("topographic__elevation", dem.astype(float), at="node", clobber=True)

grid_shape = model_grid.shape
topo_obs_2d = model_grid.at_node["topographic__elevation"].reshape(grid_shape)

# ------------------------------------------------------------
# FIX #1: Correct Y coordinate for Landlab (lower-left origin)
# Raster row=0 is TOP; Landlab y=0 is BOTTOM
# ------------------------------------------------------------
crater_x_m = col * dx_m
crater_y_m = (nrows - 1 - row) * dy_m   # <-- FIXED
center_xy_m = (crater_x_m, crater_y_m)
print("Crater center in Landlab meters:", center_xy_m)


# ============================================================
# 4) Plot observed hillshade + topo overlay
# ============================================================
hs_obs_2d = model_grid.calc_hillshade_at_node(elevs="topographic__elevation").reshape(grid_shape)

plt.figure(dpi=200, figsize=(7.6, 6.2))
plt.imshow(hs_obs_2d, cmap="gray", vmin=0, vmax=1, alpha=0.85, origin="upper")
im = plt.imshow(topo_obs_2d, cmap="terrain", alpha=0.55, origin="upper")
plt.scatter([col], [row], s=70, marker="x")
plt.title("Observed present DEM (hillshade + elevation)\nCrater center marked")
plt.colorbar(im, fraction=0.046, pad=0.02, label="Elevation (m)")
plt.tight_layout()
plt.show()


# ============================================================
# 5) Helpers: smoothing + crater
# ============================================================
def _gaussian_kernel_1d(sigma_pix):
    if sigma_pix <= 0:
        return np.array([1.0])
    radius = int(np.ceil(3 * sigma_pix))
    x = np.arange(-radius, radius + 1)
    k = np.exp(-(x**2) / (2 * sigma_pix**2))
    return k / k.sum()

def gaussian_smooth_2d(arr2d, sigma_pix):
    if sigma_pix <= 0:
        return arr2d
    k = _gaussian_kernel_1d(sigma_pix)
    pad = len(k) // 2

    tmp = np.pad(arr2d, ((0, 0), (pad, pad)), mode="edge")
    rowv = np.apply_along_axis(lambda m: np.convolve(m, k, mode="valid"), 1, tmp)

    tmp2 = np.pad(rowv, ((pad, pad), (0, 0)), mode="edge")
    colv = np.apply_along_axis(lambda m: np.convolve(m, k, mode="valid"), 0, tmp2)
    return colv

def add_crater_to_surface_m(grid, z_at_node, center_xy_m,
                            D_m, depth_m, rim_height_m,
                            rim_rel_width=0.10, soft=2.2):
    cx, cy = center_xy_m
    R = 0.5 * D_m

    x = grid.x_of_node
    y = grid.y_of_node
    r = np.sqrt((x - cx)**2 + (y - cy)**2)

    z_new = z_at_node.copy()

    inside = r <= R
    bowl = np.zeros_like(z_new)
    bowl[inside] = depth_m * (1.0 - (r[inside] / R)**soft)
    z_new -= bowl

    rim_sigma  = rim_rel_width * D_m / 2.0
    rim_center = 0.95 * R
    rim = rim_height_m * np.exp(-0.5 * ((r - rim_center) / rim_sigma)**2)
    rim_mask = (r >= 0.7 * R) & (r <= 1.2 * R)
    z_new += rim * rim_mask.astype(float)

    return z_new


# ============================================================
# 6) Prepare starting surface
# ============================================================
z_start_2d = gaussian_smooth_2d(topo_obs_2d, sigma_pix_preimpact)
z_start = z_start_2d.ravel()


# ============================================================
# 6.5) FIX #2: One fixed RMSE mask for ALL runs (baseline + crater). It's subjective. 
# ============================================================
D_mask = max(D_list)     # fixed diameter used ONLY to define mask
R_mask = 0.5 * D_mask

X0 = model_grid.x_of_node.reshape(grid_shape)
Y0 = model_grid.y_of_node.reshape(grid_shape)
mask_outside_fixed = ((X0 - center_xy_m[0])**2 + (Y0 - center_xy_m[1])**2) > (mask_mult * R_mask)**2


# ============================================================
# 7) Core model runner
# ============================================================
def run_forward(mg, fa, sp, ld, years, dt):
    if years <= 0:
        return
    n_full = int(years // dt)
    rem = years - n_full * dt

    for _ in range(n_full):
        fa.run_one_step()
        sp.run_one_step(dt)
        ld.run_one_step(dt)

    if rem > 1e-9:
        fa.run_one_step()
        sp.run_one_step(rem)
        ld.run_one_step(rem)

def simulate_to_present(impact_bp, apply_crater, D_m, depth_m, rim_height_m):
    mg = RasterModelGrid(grid_shape, xy_spacing=(dy_m, dx_m))
    mg.add_field("topographic__elevation", z_start.astype(float), at="node", clobber=True)
    mg.set_closed_boundaries_at_grid_edges(False, False, False, False)

    fa = FlowAccumulator(
        mg,
        surface="topographic__elevation",
        flow_director="D8",
        runoff_rate=None,
        depression_finder=DepressionFinderAndRouter
    )
    sp = FastscapeEroder(
        mg, K_sp=K_sp_pre, m_sp=0.5, n_sp=1.0, threshold_sp=0.0,
        discharge_field="surface_water__discharge",
        erode_flooded_nodes=False
    )
    ld = LinearDiffuser(mg, linear_diffusivity=kappa_pre)

    run_forward(mg, fa, sp, ld, years=(START_BP - impact_bp), dt=dt_years)

    if apply_crater:
        z_now = mg.at_node["topographic__elevation"]
        mg.at_node["topographic__elevation"] = add_crater_to_surface_m(
            mg, z_now, center_xy_m=center_xy_m,
            D_m=D_m, depth_m=depth_m, rim_height_m=rim_height_m,
            rim_rel_width=rim_rel_width, soft=soft
        )

    sp.K_sp = K_sp_post
    ld.linear_diffusivity = kappa_post
    run_forward(mg, fa, sp, ld, years=impact_bp, dt=dt_years)

    topo = mg.at_node["topographic__elevation"].reshape(grid_shape)

    # RMSE computed on the SAME mask for baseline and crater runs
    rmse = np.sqrt(np.mean((topo[mask_outside_fixed] - topo_obs_2d[mask_outside_fixed])**2))
    return topo, rmse


# ============================================================
# 8) Baseline RMSE (no crater) per impact time (with progress prints)
# ============================================================
baseline_rmse = {}
print("\nComputing no-crater baseline...")
for t_bp in impact_times_bp:
    print(f"  starting baseline @ {t_bp:6.0f} BP ...", flush=True)
    _, r0 = simulate_to_present(
        t_bp, apply_crater=False,
        D_m=D_mask, depth_m=0.0, rim_height_m=0.0
    )
    baseline_rmse[t_bp] = r0
    print(f"  baseline @ {t_bp:6.0f} BP: RMSE={r0:.4f} m", flush=True)


# ============================================================
# 9) Nested grid search: time × D × depth × rim_height
# ============================================================
print("\nGrid search (with crater)...")
best = {
    "impact_bp": None,
    "D_m": None,
    "depth_m": None,
    "rim_height_m": None,
    "rmse_with": np.inf,
    "rmse_baseline": None,
    "delta_rmse": None,
    "topo_with": None,
}

curves = {}

total_tests = len(impact_times_bp) * len(D_list) * len(depth_list) * len(rim_height_list)
done = 0

for D_m in D_list:
    for depth_m in depth_list:
        for rim_h in rim_height_list:
            rmse_with_curve = []
            delta_curve = []

            for t_bp in impact_times_bp:
                topo_w, r_with = simulate_to_present(
                    t_bp, apply_crater=True,
                    D_m=D_m, depth_m=depth_m, rim_height_m=rim_h
                )
                r0 = baseline_rmse[t_bp]
                rmse_with_curve.append(r_with)
                delta_curve.append(r0 - r_with)

                if r_with < best["rmse_with"]:
                    best.update({
                        "impact_bp": t_bp,
                        "D_m": D_m,
                        "depth_m": depth_m,
                        "rim_height_m": rim_h,
                        "rmse_with": r_with,
                        "rmse_baseline": r0,
                        "delta_rmse": (r0 - r_with),
                        "topo_with": topo_w
                    })

                done += 1

            curves[(D_m, depth_m, rim_h)] = {
                "rmse_with": np.array(rmse_with_curve),
                "delta": np.array(delta_curve),
            }

            print(f"  tested D={D_m:.0f} depth={depth_m:.1f} rim={rim_h:.1f}  (done {done}/{total_tests})", flush=True)


print("\nBEST RESULT (coarse scan):")
print("  impact_bp     =", best["impact_bp"])
print("  D_m           =", best["D_m"])
print("  depth_m       =", best["depth_m"])
print("  rim_height_m  =", best["rim_height_m"])
print("  RMSE with crater           =", best["rmse_with"], "m")
print("  RMSE baseline (no crater)  =", best["rmse_baseline"], "m")
print("  ΔRMSE (baseline - crater)  =", best["delta_rmse"], "m  (positive means crater helps)")


# ============================================================
# 10) Plot curves for BEST crater geometry vs baseline
# ============================================================
best_key = (best["D_m"], best["depth_m"], best["rim_height_m"])
rmse_with_best = curves[best_key]["rmse_with"]
delta_best = curves[best_key]["delta"]
rmse_base_curve = np.array([baseline_rmse[t] for t in impact_times_bp])

plt.figure(figsize=(9, 4))
plt.plot(impact_times_bp, rmse_base_curve, marker="o", label="No crater (baseline)")
plt.plot(
    impact_times_bp, rmse_with_best, marker="o",
    label=f"With crater (D={best['D_m']:.0f}, depth={best['depth_m']:.0f}, rim={best['rim_height_m']:.1f})"
)
plt.gca().invert_xaxis()
plt.xlabel("Impact time (years BP)")
plt.ylabel("RMSE vs present (m) [outside crater]")
plt.title("Timing test (coarse) for BEST crater geometry")
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(9, 4))
plt.plot(impact_times_bp, delta_best, marker="o")
plt.gca().invert_xaxis()
plt.axhline(0, linewidth=1)
plt.xlabel("Impact time (years BP)")
plt.ylabel("ΔRMSE = baseline − crater (m)")
plt.title("Does crater improve the fit? (positive is better) — BEST geometry")
plt.tight_layout()
plt.show()


# ============================================================
# 11) Best modeled present: hillshade + topo
# ============================================================
mg_best = RasterModelGrid(grid_shape, xy_spacing=(dy_m, dx_m))
mg_best.add_field("topographic__elevation", best["topo_with"].ravel(), at="node", clobber=True)
hs_best = mg_best.calc_hillshade_at_node(elevs="topographic__elevation").reshape(grid_shape)

plt.figure(dpi=200, figsize=(7.6, 6.2))
plt.imshow(hs_best, cmap="gray", vmin=0, vmax=1, alpha=0.85, origin="upper")
im = plt.imshow(best["topo_with"], cmap="terrain", alpha=0.55, origin="upper")
plt.title(
    f"Best modeled present (impact at {best['impact_bp']:.0f} BP)\n"
    f"D={best['D_m']:.0f} m, depth={best['depth_m']:.0f} m, rim={best['rim_height_m']:.1f} m"
)
plt.colorbar(im, fraction=0.046, pad=0.02, label="Elevation (m)")
plt.tight_layout()
plt.show()


# ============================================================
# 12) Difference maps
# ============================================================
delta_map = best["topo_with"] - topo_obs_2d

plt.figure(dpi=200, figsize=(7.6, 3.8))
im = plt.imshow(delta_map, cmap="RdBu_r", origin="upper")
plt.title("Difference: Best modeled present − Observed present (m)")
plt.colorbar(im, fraction=0.046, pad=0.02, label="Elevation difference (m)")
plt.tight_layout()
plt.show()

delta_outside = delta_map.copy()
delta_outside[~mask_outside_fixed] = np.nan

plt.figure(dpi=200, figsize=(7.6, 3.8))
im = plt.imshow(delta_outside, cmap="RdBu_r", origin="upper")
plt.title("Difference OUTSIDE crater only (modeled − observed)")
plt.colorbar(im, fraction=0.046, pad=0.02, label="Elevation difference (m)")
plt.tight_layout()
plt.show()
